In [1]:
import yt_dlp
import os
import re

# Function to clean filenames by stripping unwanted characters
def clean_filename(filename):
    cleaned = re.sub(r'[\\/:\*?"<>|\.]+', '', filename) 
    cleaned = re.sub(r'[^\w\-_\.]', '', cleaned)  
    return cleaned.strip() 

# Function to download audio, manual and auto-generated sub files and extracting the file names
def ydl_download(url, auto_sub_path='auto_sub/', manual_sub_path='manual_sub/', audio_path='audio_files/'):
    # Gotta make sure the paths for auto subs, manual subs, and audio files exist or create them
    os.makedirs(auto_sub_path, exist_ok=True)
    os.makedirs(manual_sub_path, exist_ok=True)
    os.makedirs(audio_path, exist_ok=True)

    # Options just to extract the video title
    title_extract_opts = {
        'quiet': True,                     
        'skip_download': True,                             
    }
    
    # Options for downloading manually submitted subtitles
    manual_sub_opts = {
        'writesubtitles': True,             
        'skip_download': True,              
        'subtitlesformat': 'vtt',           
    }

    # Options for downloading auto-generated subtitles
    auto_sub_opts = {
        'writeautomaticsub': True,          
        'skip_download': True,              
        'subtitlesformat': 'vtt',           
        'subtitleslangs': ['en'],           
    }

    # Options for downloading audio
    audio_opts = {
        'format': 'bestaudio/best/worstaudio',         
    }

    # Download manually submitted subtitles
    with yt_dlp.YoutubeDL(title_extract_opts) as ydl_title:
        info_dict = ydl_title.extract_info(url)
        video_title = info_dict.get('title', None)
        cleaned_title = clean_filename(video_title)  # Clean the video title for use in file names

    # Update the output template with the cleaned title
    manual_sub_opts['outtmpl'] = os.path.join(manual_sub_path, f'{cleaned_title}_manual.%(ext)s')
    auto_sub_opts['outtmpl'] = os.path.join(auto_sub_path, f'{cleaned_title}_auto.%(ext)s')
    audio_opts['outtmpl'] = os.path.join(audio_path, f'{cleaned_title}.%(ext)s')

    # Now download the manually submitted subtitles with updated options
    with yt_dlp.YoutubeDL(manual_sub_opts) as ydl_manual:
        ydl_manual.extract_info(url)

    # Download auto-generated subtitles
    with yt_dlp.YoutubeDL(auto_sub_opts) as ydl_auto:
        ydl_auto.extract_info(url)

    # Download audio and capture the info
    with yt_dlp.YoutubeDL(audio_opts) as ydl_audio:
        audio_info = ydl_audio.extract_info(url, download=True)

    # Get the downloaded audio file name and extension
    audio_filename = ydl_audio.prepare_filename(audio_info)
    audio_extension = audio_filename.split('.')[-1]
    
    return cleaned_title, audio_filename

In [2]:
import webvtt

# Generating the different subtitle files and the audio file using the ydl_download function and return the file names for the next function
def generate_subtitles(url):
    title, audio = ydl_download(url)
    audio_title = audio
    return title, audio_title
# video_name, audio_name = generate_subtitles('https://www.youtube.com/watch?v=PxCxlsl_YwY')

In [3]:
video_name, audio_name

NameError: name 'video_name' is not defined

In [4]:
import yt_dlp

# URL of the YouTube playlist
playlist_urls = ['https://www.youtube.com/playlist?list=PLUl4u3cNGP61O7HkcF7UImpM0cR_L2gSw', 'https://www.youtube.com/playlist?list=PL4C4C8A7D06566F38']

# Function to download playlist info and extract the video urls to store them in a list
def playlist_downloads(pl_url):
    ydl_pl_opts = {
        'quiet': True,
        'extract_flat': True, 
        'skip_download': True
    }

    # Use yt-dlp to extract the playlist information
    with yt_dlp.YoutubeDL(ydl_pl_opts) as ydl:
        playlist_info = ydl.extract_info(pl_url, download=False)

    # Extract video URLs
    pl_video_urls = [f"https://www.youtube.com/watch?v={entry['id']}" for entry in playlist_info['entries']]
    
    return pl_video_urls

# List to store videos
pl_videos = []

# # For loop to get all the URLs
# for pl_url in playlist_urls:
#     pl_videos+=playlist_downloads(pl_url)
# pl_videos
pl_videos = playlist_downloads(playlist_urls[1])
pl_videos

['https://www.youtube.com/watch?v=PxCxlsl_YwY',
 'https://www.youtube.com/watch?v=9FLItlbBUPY',
 'https://www.youtube.com/watch?v=bHdzkFrgRcA',
 'https://www.youtube.com/watch?v=YBajUR3EFSM',
 'https://www.youtube.com/watch?v=57jzPlxf4fk',
 'https://www.youtube.com/watch?v=0D4BbCa4gHo',
 'https://www.youtube.com/watch?v=U1EcnfTKXJ0',
 'https://www.youtube.com/watch?v=dK3NEf13nPc',
 'https://www.youtube.com/watch?v=UYe98CcxPbs',
 'https://www.youtube.com/watch?v=3_goGnJm5sA',
 'https://www.youtube.com/watch?v=7eZVshlT33Q',
 'https://www.youtube.com/watch?v=2XraaWefBd8',
 'https://www.youtube.com/watch?v=15HVevXRsBA',
 'https://www.youtube.com/watch?v=23xbkrpQuAo',
 'https://www.youtube.com/watch?v=ChiM2-MV-qM',
 'https://www.youtube.com/watch?v=YP_B0AapU0c',
 'https://www.youtube.com/watch?v=60e4hdCi1D4',
 'https://www.youtube.com/watch?v=UZb9hZIAvL4',
 'https://www.youtube.com/watch?v=xrypSZU8cBE',
 'https://www.youtube.com/watch?v=o7UCBjGsRTE',
 'https://www.youtube.com/watch?v=z5TPjZ

In [5]:
len(pl_videos)

35

In [6]:
import whisper
import os

def audio_transcription(video_title, audio_file, whisper_sub_path='whisper_sub/'):
    #Making sure the Whisper subtitle path exists
    os.makedirs(whisper_sub_path, exist_ok=True)
    
    # Load the Whisper model
    model = whisper.load_model('base')
    # Transcribe the audio file with word timestamps
    result = model.transcribe(audio_file, word_timestamps=True)
    # Define the subtitle file name
    sub_file = video_title + '.vtt'
    sub_file = os.path.join(whisper_sub_path, sub_file)
    
    # Open the VTT file for writing
    with open(sub_file, "w", encoding="utf-8") as vtt:
        vtt.write("WEBVTT\n\n")  # Write the VTT header

        for segment in result['segments']:
            # Start time conversion of each segment
            time_start = segment['start']
            start_hour = int(time_start // 3600)
            start_min = int((time_start % 3600) // 60)
            start_sec = int(time_start % 60)
            start_msec = round(time_start % 1, 3) * 1000  
            # End time conversion of each segment
            time_end = segment['end']
            end_hour = int(time_end // 3600)
            end_min = int((time_end % 3600) // 60)
            end_sec = int(time_end % 60)
            end_msec = round(time_end % 1, 3) * 1000  

            # Write the timestamp in the format required by VTT
            vtt.write(f"{start_hour:02}:{start_min:02}:{start_sec:02}.{int(start_msec):03} --> "
                       f"{end_hour:02}:{end_min:02}:{end_sec:02}.{int(end_msec):03}\n")
            vtt.write(f"{segment['text'].strip()}\n\n")


In [7]:
# For loop to go through the URLs and generate the audio transcriptions
for video in pl_videos:
    video_title, audio_title = generate_subtitles(video)
    audio_transcription(video_title, audio_title)
    print(f"\nTranscribed the file: {video_title}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=PxCxlsl_YwY
[youtube] PxCxlsl_YwY: Downloading webpage
[youtube] PxCxlsl_YwY: Downloading ios player API JSON
[youtube] PxCxlsl_YwY: Downloading web creator player API JSON
[youtube] PxCxlsl_YwY: Downloading m3u8 information
[info] PxCxlsl_YwY: Downloading subtitles: en
[info] PxCxlsl_YwY: Downloading 1 format(s): 244+251
Deleting existing file manual_sub/Lec1DotproductMIT1802MultivariableCalculusFall2007_manual.en.vtt
[info] Writing video subtitles to: manual_sub/Lec1DotproductMIT1802MultivariableCalculusFall2007_manual.en.vtt
[download] Destination: manual_sub/Lec1DotproductMIT1802MultivariableCalculusFall2007_manual.en.vtt
[download] 100% of   38.24KiB in 00:00:00 at 522.03KiB/s
[youtube] Extracting URL: https://www.youtube.com/watch?v=PxCxlsl_YwY
[youtube] PxCxlsl_YwY: Downloading webpage
[youtube] PxCxlsl_YwY: Downloading ios player API JSON
[youtube] PxCxlsl_YwY: Downloading web creator player API JSON
[youtube] PxCxlsl_Yw

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Transcribed the file: Lec1DotproductMIT1802MultivariableCalculusFall2007
[youtube] Extracting URL: https://www.youtube.com/watch?v=9FLItlbBUPY
[youtube] 9FLItlbBUPY: Downloading webpage
[youtube] 9FLItlbBUPY: Downloading ios player API JSON
[youtube] 9FLItlbBUPY: Downloading web creator player API JSON
[youtube] 9FLItlbBUPY: Downloading m3u8 information
[info] 9FLItlbBUPY: Downloading 1 format(s): 244+251
[info] There are no subtitles for the requested languages
[youtube] Extracting URL: https://www.youtube.com/watch?v=9FLItlbBUPY
[youtube] 9FLItlbBUPY: Downloading webpage
[youtube] 9FLItlbBUPY: Downloading ios player API JSON
[youtube] 9FLItlbBUPY: Downloading web creator player API JSON
[youtube] 9FLItlbBUPY: Downloading m3u8 information
[info] 9FLItlbBUPY: Downloading subtitles: en
[info] 9FLItlbBUPY: Downloading 1 format(s): 244+251
[info] Writing video subtitles to: auto_sub/Lec2DeterminantscrossproductMIT1802MultivariableCalculusFall2007_auto.en.vtt
[download] Destination: auto_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Transcribed the file: Lec2DeterminantscrossproductMIT1802MultivariableCalculusFall2007
[youtube] Extracting URL: https://www.youtube.com/watch?v=bHdzkFrgRcA
[youtube] bHdzkFrgRcA: Downloading webpage
[youtube] bHdzkFrgRcA: Downloading ios player API JSON
[youtube] bHdzkFrgRcA: Downloading web creator player API JSON
[youtube] bHdzkFrgRcA: Downloading m3u8 information
[info] bHdzkFrgRcA: Downloading subtitles: en-qlPKC2UN_YU
[info] bHdzkFrgRcA: Downloading 1 format(s): 244+251
[info] Writing video subtitles to: manual_sub/Lec3MatricesinversematricesMIT1802MultivariableCalculusFall2007_manual.en-qlPKC2UN_YU.vtt
[download] Destination: manual_sub/Lec3MatricesinversematricesMIT1802MultivariableCalculusFall2007_manual.en-qlPKC2UN_YU.vtt
[download] 100% of   46.98KiB in 00:00:00 at 642.52KiB/s
[youtube] Extracting URL: https://www.youtube.com/watch?v=bHdzkFrgRcA
[youtube] bHdzkFrgRcA: Downloading webpage
[youtube] bHdzkFrgRcA: Downloading ios player API JSON
[youtube] bHdzkFrgRcA: Downloadi

...


Transcribed the file: Lec20PathindependenceandconservativefieldsMIT1802MultivariableCalculusFall2007
[youtube] Extracting URL: https://www.youtube.com/watch?v=z5TPjZrsp2k
[youtube] z5TPjZrsp2k: Downloading webpage
[youtube] z5TPjZrsp2k: Downloading ios player API JSON
[youtube] z5TPjZrsp2k: Downloading web creator player API JSON
[youtube] z5TPjZrsp2k: Downloading m3u8 information
[info] z5TPjZrsp2k: Downloading subtitles: en-qlPKC2UN_YU
[info] z5TPjZrsp2k: Downloading 1 format(s): 135+140
[info] Writing video subtitles to: manual_sub/Lec21GradientfieldsandpotentialfunctionsMIT1802MultivariableCalculusFall2007_manual.en-qlPKC2UN_YU.vtt
[download] Destination: manual_sub/Lec21GradientfieldsandpotentialfunctionsMIT1802MultivariableCalculusFall2007_manual.en-qlPKC2UN_YU.vtt
[download] 100% of   43.49KiB in 00:00:00 at 439.31KiB/s
[youtube] Extracting URL: https://www.youtube.com/watch?v=z5TPjZrsp2k
[youtube] z5TPjZrsp2k: Downloading webpage
[youtube] z5TPjZrsp2k: Downloading ios player A

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Transcribed the file: Lec21GradientfieldsandpotentialfunctionsMIT1802MultivariableCalculusFall2007
[youtube] Extracting URL: https://www.youtube.com/watch?v=tYdoS0tkAHA
[youtube] tYdoS0tkAHA: Downloading webpage
[youtube] tYdoS0tkAHA: Downloading ios player API JSON
[youtube] tYdoS0tkAHA: Downloading web creator player API JSON
[youtube] tYdoS0tkAHA: Downloading m3u8 information
[info] tYdoS0tkAHA: Downloading subtitles: en-qlPKC2UN_YU
[info] tYdoS0tkAHA: Downloading 1 format(s): 135+140
[info] Writing video subtitles to: manual_sub/Lec22GreenstheoremMIT1802MultivariableCalculusFall2007_manual.en-qlPKC2UN_YU.vtt
[download] Destination: manual_sub/Lec22GreenstheoremMIT1802MultivariableCalculusFall2007_manual.en-qlPKC2UN_YU.vtt
[download] 100% of   41.95KiB in 00:00:00 at 310.41KiB/s
[youtube] Extracting URL: https://www.youtube.com/watch?v=tYdoS0tkAHA
[youtube] tYdoS0tkAHA: Downloading webpage
[youtube] tYdoS0tkAHA: Downloading ios player API JSON
[youtube] tYdoS0tkAHA: Downloading web

[download] Got error: HTTPSConnectionPool(host='rr5---sn-vgqskn67.googlevideo.com', port=443): Read timed out.


DownloadError: ERROR: [download] Got error: HTTPSConnectionPool(host='rr5---sn-vgqskn67.googlevideo.com', port=443): Read timed out.